In [ ]:
import os
import json
import shutil
from pathlib import Path    
import random

In [4]:
import os
import json
import shutil
from pathlib import Path    
import random
from PIL import Image
from tqdm import tqdm

def create_coco_structure():
    # Define paths
    base_dir = Path("data")
    data_dir = base_dir
    
    # Create directory structure
    dirs = [
        data_dir / "annotations",
        data_dir / "train",
        data_dir / "val",
        data_dir / "test",
    ]
    
    for dir_path in dirs:
        dir_path.mkdir(parents=True, exist_ok=True)
    
    return data_dir

def yolo_to_coco_bbox(yolo_bbox, img_width, img_height):
    """Convert YOLO bbox to COCO bbox format [x_min, y_min, width, height]"""
    x_center, y_center, width, height = yolo_bbox
    
    # Convert normalized coordinates to absolute coordinates
    x_center = float(x_center) * img_width
    y_center = float(y_center) * img_height
    width = float(width) * img_width
    height = float(height) * img_height
    
    # Convert from center coordinates to top-left coordinates
    x_min = x_center - (width / 2)
    y_min = y_center - (height / 2)
    
    return [x_min, y_min, width, height]

def create_coco_annotations_from_yolo(split_dir, split="train"):
    coco_format = {
        "images": [],
        "annotations": [],
        "categories": [
            {"id": 1, "name": "fire"},
            {"id": 2, "name": "smoke"}
        ]
    }
    
    image_dir = split_dir / "images"
    label_dir = split_dir / "labels"
    annotation_id = 1
    
    # Process each image and its corresponding label file
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    for img_id, img_file in enumerate(tqdm(image_files, desc=f"Processing {split} annotations"), 1):
        # Get image dimensions
        img_path = image_dir / img_file
        with Image.open(img_path) as img:
            img_width, img_height = img.size
        
        # Add image info
        image_info = {
            "id": img_id,
            "file_name": img_file,
            "height": img_height,
            "width": img_width
        }
        coco_format["images"].append(image_info)
        
        # Process corresponding label file
        label_file = label_dir / f"{os.path.splitext(img_file)[0]}.txt"
        if label_file.exists():
            with open(label_file, 'r') as f:
                for line in f:
                    class_id, x_center, y_center, width, height = map(float, line.strip().split())
                    
                    # Convert YOLO bbox to COCO bbox
                    bbox = yolo_to_coco_bbox([x_center, y_center, width, height], img_width, img_height)
                    
                    # Create annotation
                    annotation = {
                        "id": annotation_id,
                        "image_id": img_id,
                        "category_id": int(class_id) + 1,  # YOLO uses 0-based indices
                        "bbox": bbox,
                        "area": bbox[2] * bbox[3],
                        "iscrowd": 0
                    }
                    coco_format["annotations"].append(annotation)
                    annotation_id += 1
    
    return coco_format

def main():
    # Create directory structure
    data_dir = create_coco_structure()
    
    # Source directories
    source_base = Path("../data")  # Your current data directory
    splits = ["train", "val", "test"]
    
    for split in tqdm(splits, desc="Processing splits"):
        # Create COCO annotations
        split_dir = source_base / split
        coco_annotations = create_coco_annotations_from_yolo(split_dir, split)
        
        # Copy images to new structure
        source_images = split_dir / "images"
        target_images = data_dir / split
        
        # Copy all images
        image_files = [f for f in os.listdir(source_images) if f.endswith(('.jpg', '.jpeg', '.png'))]
        for img_file in tqdm(image_files, desc=f"Copying {split} images"):
            shutil.copy2(source_images / img_file, target_images / img_file)
        
        # Save annotations
        with open(data_dir / "annotations" / f"instances_{split}.json", 'w') as f:
            json.dump(coco_annotations, f)
        print(f"Saved {split} annotations")

if __name__ == "__main__":
    main()

Processing splits:  33%|███▎      | 1/3 [00:18<00:37, 18.61s/it]

Saved train annotations


Processing splits:  67%|██████▋   | 2/3 [00:22<00:09,  9.95s/it]

Saved val annotations


Processing splits: 100%|██████████| 3/3 [00:29<00:00,  9.69s/it]

Saved test annotations


In [ ]:
# # New cell to download configs
# !mim download mmdet --config rtmdet_tiny_8xb32-300e_coco --dest .

processing rtmdet_tiny_8xb32-300e_coco...
downloading ---------------------------------------- 0.0/54.9 MiB ? eta -:--:--
downloading --------------------------------- 0.0/54.9 MiB 1.1 MB/s eta 0:00:53
downloading --------------------------------- 0.1/54.9 MiB 1.5 MB/s eta 0:00:40
downloading --------------------------------- 0.4/54.9 MiB 3.0 MB/s eta 0:00:19
downloading --------------------------------- 0.5/54.9 MiB 3.3 MB/s eta 0:00:18
downloading --------------------------------- 0.7/54.9 MiB 3.7 MB/s eta 0:00:16
downloading  -------------------------------- 0.9/54.9 MiB 3.6 MB/s eta 0:00:16
downloading  -------------------------------- 1.2/54.9 MiB 5.5 MB/s eta 0:00:11
downloading  -------------------------------- 1.4/54.9 MiB 5.5 MB/s eta 0:00:11
downloading - ------------------------------- 1.7/54.9 MiB 5.4 MB/s eta 0:00:11
downloading - ------------------------------- 1.9/54.9 MiB 5.4 MB/s eta 0:00:11
downloading - ------------------------------- 2.2/54.9 MiB 5.9 MB/s eta 0:00:1

In [1]:
!mim train mmdet custom_rtmdet.py 

^C


Training command is C:\Users\Akshat\anaconda3\envs\mmdet_env\python.exe c:\users\akshat\anaconda3\envs\mmdet_env\lib\site-packages\mmdet\.mim\tools\train.py custom_rtmdet.py --launcher none. 
04/20 13:47:36 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: win32
    Python: 3.10.16 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:19:12) [MSC v.1929 64 bit (AMD64)]
    CUDA available: False
    MUSA available: False
    numpy_random_seed: 1715166096
    MSVC: Microsoft (R) C/C++ Optimizing Compiler Version 19.43.34810 for x64
    GCC: n/a
    PyTorch: 2.6.0+cpu
    PyTorch compiling details: PyTorch built with:
  - C++ Version: 201703
  - MSVC 192930157
  - Intel(R) oneAPI Math Kernel Library Version 2025.0.1-Product Build 20241031 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.5.3 (Git Hash 66f0cb9eb66affd2da3bf5f8d897376f04aae6af)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL

C:\Users\Akshat\anaconda3\envs\mmdet_env\lib\site-packages\mmdet\models\layers\se_layer.py:158: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
C:\Users\Akshat\anaconda3\envs\mmdet_env\lib\site-packages\mmdet\models\backbones\csp_darknet.py:118: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
C:\Users\Akshat\anaconda3\envs\mmdet_env\lib\site-packages\torch\functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:3638.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
